# MARKET BASKET ANALYSIS ANALYSIS USING: FP-GROWTH MODEL

## Problem Statement: 
Implementation of Market Basket Analysis using FP-Growth model on grocery store invoice basket for removing frequent itemsets from the dataset to determine the shopping patterns of a customer.

## Details of the Data:
+ DATA SOURCE: Github 
+ DATA DICTIONARY: Each row consists of items present in a single customer's basket.

## Market Basket Analysis:
Market Basket Analysis is a modelling technique based upon the theory that if you buy a certain group of items, you are more (or less) likely to buy another group of items. For example, if you are in an English pub and you buy a pint of beer and don't buy a bar meal, you are more likely to buy crisps (US. chips) at the same time than somebody who didn't buy beer.

The set of items a customer buys is referred to as an itemset, and market basket analysis seeks to find relationships between purchases.

Typically the relationship will be in the form of a rule:

IF {beer, no bar meal} THEN {crisps}.
The probability that a customer will buy beer without a bar meal (i.e. that the antecedent is true) is referred to as the support for the rule. The conditional probability that a customer will purchase crisps is referred to as the confidence.

## Algorithm used: FP Growth:
A FP-tree is a compact data structure that represents the data set in tree form.  Each transaction is read and then mapped onto a path in the FP-tree. This is done until all transactions have been read. Different transactions that have common subsets allow the tree to remain compact because their paths overlap.

The diagram to the right is an example of a best-case scenario that occurs when all transactions have exactly the same itemset; the size of the FP-tree will be only a single branch of nodes. 
The worst case scenario occurs when every transaction has a unique itemset and so the space needed to store the tree is greater than the space used to store the original data set because the FP-tree requires additional space to store pointers between nodes and also the counters for each item. The diagram below is an example of how a worst case scenario FP-tree might appear. As you can see, the complexity of the tree grows with the uniqueness of each transaction.

**Construction**

The construction of a FP-tree is subdivided into three major steps:

1. Scan the data set to determine the support count of each item, discard the infrequent items and sort the frequent items in decreasing order

2. Scan the data set one transaction at a time to create the FP-tree. For each transaction:

+ If it is a unique transaction form a new path and set the counter for each node to 1.
+ If it shares a common prefix itemset then increment the common itemset node counters and create new nodes if needed.
+ Continue this until each transaction has been mapped unto the tree.
<img src='./fptree.jpg' />  

## Exploratory Data Analysis:
+ **Data size** - 9835 rows with 163 unique values
+ **Null value count** - Zero
+ **Number of attributes** - 163 Unique values


## Flow of the Program:
1. Importing Packages
2. Data Pre-processing
3. Creation Of Transactions
4. Building the model
5. Calculation of different metrics

## Evaluation of the model:

Association rules analysis is a technique to uncover how items are associated to each other. There are three common ways to measure association.

+ Measure 1: Support. This says how popular an itemset is, as measured by the proportion of transactions in which an itemset appears. In Table 1 below, the support of {apple} is 4 out of 8, or 50%. Itemsets can also contain multiple items. For instance, the support of {apple, beer, rice} is 2 out of 8, or 25%.

<img src='1.png' />  

If you discover that sales of items beyond a certain proportion tend to have a significant impact on your profits, you might consider using that proportion as your support threshold. You may then identify itemsets with support values above this threshold as significant itemsets.

+ Measure 2: Confidence. This says how likely item Y is purchased when item X is purchased, expressed as {X -> Y}. This is measured by the proportion of transactions with item X, in which item Y also appears. In Table 1, the confidence of {apple -> beer} is 3 out of 4, or 75%.

<img src='./2.png' />  

One drawback of the confidence measure is that it might misrepresent the importance of an association. This is because it only accounts for how popular apples are, but not beers. If beers are also very popular in general, there will be a higher chance that a transaction containing apples will also contain beers, thus inflating the confidence measure. To account for the base popularity of both constituent items, we use a third measure called lift.

+ Measure 3: Lift. This says how likely item Y is purchased when item X is purchased, while controlling for how popular item Y is. In Table 1, the lift of {apple -> beer} is 1,which implies no association between items. A lift value greater than 1 means that item Y is likely to be bought if item X is bought, while a value less than 1 means that item Y is unlikely to be bought if item X is bought.

<img src='./3.png' />  

Recall that one drawback of the confidence measure is that it tends to misrepresent the importance of an association. To demonstrate this, we go back to the main dataset to pick 3 association rules containing beer:

<img src='./4.png' />  

The {beer -> soda} rule has the highest confidence at 20%. However, both beer and soda appear frequently across all transactions (see Table 3), so their association could simply be a fluke. This is confirmed by the lift value of {beer -> soda}, which is 1, implying no association between beer and soda.

<img src='./5.png' />  

On the other hand, the {beer -> male cosmetics} rule has a low confidence, due to few purchases of male cosmetics in general. However, whenever someone does buy male cosmetics, he is very likely to buy beer as well, as inferred from a high lift value of 2.6. The converse is true for {beer -> berries}. With a lift value below 1, we may conclude that if someone buys berries, he would likely be averse to beer.


## 1.  Importing packages:

In [30]:
#importing packages
from pyspark.sql.functions import *
from  pyspark.sql import SQLContext 
from pyspark.sql import SparkSession
from pyspark.conf import SparkConf
from pyspark import SparkContext 
import pyspark.sql.types as typ
from itertools import combinations
import numpy as np
import pandas as pd

In [2]:
#creating a spark session
spark=SparkSession.builder.appName("Neeha").config(conf=SparkConf()).getOrCreate()

## 2. Data preprocessing:

In [3]:
#importing the fpgrowth model
from pyspark.mllib.fpm import FPGrowth, FPGrowthModel
#importing dataset into rdd
new = sc.textFile("file:////home/dsuser/qlabs/dev_NeehaKhalfay/Market_Basket_Example/groc_comma.txt")
new.take(5)


[u'citrus fruit,semi-finished bread,margarine,ready soups', u'tropical fruit,yogurt,coffee', u'whole milk', u'pip fruit,yogurt,cream cheese ,meat spreads', u'other vegetables,whole milk,condensed milk,long life bakery product']

In [4]:
#replacing spaces between a single item with a hiphen so that the algorithm may take it as a single item and not 2 items
new2 = new.map(lambda x : x.replace(' ', '-'))

In [5]:
#replacing a comma with a space to determine that the entire record is a single basket
new3 = new2.map(lambda x : x.replace(',', ' '))

In [6]:
new3.take(5)

[u'citrus-fruit semi-finished-bread margarine ready-soups', u'tropical-fruit yogurt coffee', u'whole-milk', u'pip-fruit yogurt cream-cheese- meat-spreads', u'other-vegetables whole-milk condensed-milk long-life-bakery-product']

## 3. Creation of transactions:

In [7]:
#Creation of transactions in a specified format for the model to understand from the baskets
transactions = new3.map(lambda line: line.strip().split(' '))
transactions.take(5)

[[u'citrus-fruit', u'semi-finished-bread', u'margarine', u'ready-soups'], [u'tropical-fruit', u'yogurt', u'coffee'], [u'whole-milk'], [u'pip-fruit', u'yogurt', u'cream-cheese-', u'meat-spreads'], [u'other-vegetables', u'whole-milk', u'condensed-milk', u'long-life-bakery-product']]

## 4. Building the model:

In [8]:
#Building the model
model_fp = FPGrowth.train(transactions, minSupport=0.007, numPartitions=10)

In [9]:
#storing the frequent itemsets in the result and displaying them
result = model_fp.freqItemsets().collect()
for fi in result:
    print(fi)

FreqItemset(items=[u'ham'], freq=256)
FreqItemset(items=[u'ham', u'other-vegetables'], freq=90)
FreqItemset(items=[u'ham', u'whole-milk'], freq=113)
FreqItemset(items=[u'liquor'], freq=109)
FreqItemset(items=[u'frankfurter'], freq=580)
FreqItemset(items=[u'frankfurter', u'shopping-bags'], freq=81)
FreqItemset(items=[u'frankfurter', u'rolls/buns'], freq=189)
FreqItemset(items=[u'frankfurter', u'bottled-water'], freq=72)
FreqItemset(items=[u'frankfurter', u'yogurt'], freq=110)
FreqItemset(items=[u'frankfurter', u'tropical-fruit'], freq=93)
FreqItemset(items=[u'frankfurter', u'pastry'], freq=82)
FreqItemset(items=[u'frankfurter', u'other-vegetables'], freq=162)
FreqItemset(items=[u'frankfurter', u'other-vegetables', u'whole-milk'], freq=75)
FreqItemset(items=[u'frankfurter', u'domestic-eggs'], freq=69)
FreqItemset(items=[u'frankfurter', u'brown-bread'], freq=70)
FreqItemset(items=[u'frankfurter', u'sausage'], freq=99)
FreqItemset(items=[u'frankfurter', u'soda'], freq=111)
FreqItemset(item

FreqItemset(items=[u'pasta'], freq=148)
FreqItemset(items=[u'bottled-beer'], freq=792)
FreqItemset(items=[u'bottled-beer', u'rolls/buns'], freq=134)
FreqItemset(items=[u'bottled-beer', u'bottled-water'], freq=155)
FreqItemset(items=[u'bottled-beer', u'yogurt'], freq=91)
FreqItemset(items=[u'bottled-beer', u'tropical-fruit'], freq=81)
FreqItemset(items=[u'bottled-beer', u'other-vegetables'], freq=159)
FreqItemset(items=[u'bottled-beer', u'other-vegetables', u'whole-milk'], freq=75)
FreqItemset(items=[u'bottled-beer', u'sausage'], freq=77)
FreqItemset(items=[u'bottled-beer', u'soda'], freq=167)
FreqItemset(items=[u'bottled-beer', u'root-vegetables'], freq=95)
FreqItemset(items=[u'bottled-beer', u'whole-milk'], freq=201)
FreqItemset(items=[u'instant-coffee'], freq=73)
FreqItemset(items=[u'onions'], freq=305)
FreqItemset(items=[u'onions', u'yogurt'], freq=71)
FreqItemset(items=[u'onions', u'other-vegetables'], freq=140)
FreqItemset(items=[u'onions', u'root-vegetables'], freq=93)
FreqItemse

FreqItemset(items=[u'bottled-water', u'yogurt', u'whole-milk'], freq=95)
FreqItemset(items=[u'bottled-water', u'other-vegetables'], freq=244)
FreqItemset(items=[u'bottled-water', u'other-vegetables', u'whole-milk'], freq=106)
FreqItemset(items=[u'bottled-water', u'soda'], freq=285)
FreqItemset(items=[u'bottled-water', u'soda', u'whole-milk'], freq=74)
FreqItemset(items=[u'bottled-water', u'whole-milk'], freq=338)
FreqItemset(items=[u'hard-cheese'], freq=241)
FreqItemset(items=[u'hard-cheese', u'other-vegetables'], freq=93)
FreqItemset(items=[u'hard-cheese', u'whole-milk'], freq=99)
FreqItemset(items=[u'flower-(seeds)'], freq=102)
FreqItemset(items=[u'curd'], freq=524)
FreqItemset(items=[u'curd', u'whipped/sour-cream'], freq=103)
FreqItemset(items=[u'curd', u'citrus-fruit'], freq=70)
FreqItemset(items=[u'curd', u'rolls/buns'], freq=99)
FreqItemset(items=[u'curd', u'yogurt'], freq=170)
FreqItemset(items=[u'curd', u'yogurt', u'whole-milk'], freq=99)
FreqItemset(items=[u'curd', u'tropical-

FreqItemset(items=[u'whipped/sour-cream', u'root-vegetables', u'other-vegetables'], freq=84)
FreqItemset(items=[u'whipped/sour-cream', u'root-vegetables', u'whole-milk'], freq=93)
FreqItemset(items=[u'whipped/sour-cream', u'pip-fruit'], freq=91)
FreqItemset(items=[u'whipped/sour-cream', u'whole-milk'], freq=317)
FreqItemset(items=[u'oil'], freq=276)
FreqItemset(items=[u'oil', u'other-vegetables'], freq=98)
FreqItemset(items=[u'oil', u'root-vegetables'], freq=69)
FreqItemset(items=[u'oil', u'whole-milk'], freq=111)
FreqItemset(items=[u'pot-plants'], freq=170)
FreqItemset(items=[u'house-keeping-products'], freq=82)
FreqItemset(items=[u'tropical-fruit'], freq=1032)
FreqItemset(items=[u'tropical-fruit', u'rolls/buns'], freq=242)
FreqItemset(items=[u'tropical-fruit', u'rolls/buns', u'other-vegetables'], freq=77)
FreqItemset(items=[u'tropical-fruit', u'rolls/buns', u'whole-milk'], freq=108)
FreqItemset(items=[u'tropical-fruit', u'bottled-water'], freq=182)
FreqItemset(items=[u'tropical-fruit

FreqItemset(items=[u'domestic-eggs', u'other-vegetables', u'whole-milk'], freq=121)
FreqItemset(items=[u'domestic-eggs', u'sausage'], freq=94)
FreqItemset(items=[u'domestic-eggs', u'soda'], freq=122)
FreqItemset(items=[u'domestic-eggs', u'pip-fruit'], freq=85)
FreqItemset(items=[u'domestic-eggs', u'root-vegetables'], freq=141)
FreqItemset(items=[u'domestic-eggs', u'root-vegetables', u'other-vegetables'], freq=72)
FreqItemset(items=[u'domestic-eggs', u'root-vegetables', u'whole-milk'], freq=84)
FreqItemset(items=[u'domestic-eggs', u'whole-milk'], freq=295)
FreqItemset(items=[u'processed-cheese'], freq=163)
FreqItemset(items=[u'processed-cheese', u'whole-milk'], freq=69)
FreqItemset(items=[u'Instant-food-products'], freq=79)
FreqItemset(items=[u'sausage'], freq=924)
FreqItemset(items=[u'sausage', u'shopping-bags'], freq=154)
FreqItemset(items=[u'sausage', u'rolls/buns'], freq=301)
FreqItemset(items=[u'sausage', u'rolls/buns', u'other-vegetables'], freq=87)
FreqItemset(items=[u'sausage', 

In [11]:
#Converting the results into a dataframe
dfp = pd.DataFrame(result)
dfp

                                            items  freq
0                                           [ham]   256
1                         [ham, other-vegetables]    90
2                               [ham, whole-milk]   113
3                                        [liquor]   109
4                                   [frankfurter]   580
5                    [frankfurter, shopping-bags]    81
6                       [frankfurter, rolls/buns]   189
7                    [frankfurter, bottled-water]    72
8                           [frankfurter, yogurt]   110
9                   [frankfurter, tropical-fruit]    93
10                          [frankfurter, pastry]    82
11                [frankfurter, other-vegetables]   162
12    [frankfurter, other-vegetables, whole-milk]    75
13                   [frankfurter, domestic-eggs]    69
14                     [frankfurter, brown-bread]    70
15                         [frankfurter, sausage]    99
16                            [frankfurter, soda

In [26]:
#converting the frequencies to support

def floatsup(x, y):
    """
    Converts the frequency of a number to support by dividing it with the total no. of rows.
     Parameters
    -----------
    x: series of frequencies
    y: total no. of rows in a dataset.
    Returns
    ----------
    returns x/y
    
    """
    return float(x)/y

In [13]:
abc = new.count() #No of rows in a dataset
abc = float(abc)
abc

9835.0

In [27]:
freq_ser =dfp['freq']
freq_ser_float = freq_ser.apply(floatsup, args=(abc,)) # method to convert to support

In [28]:
freq_ser_float

0      0.026029
1      0.009151
2      0.011490
3      0.011083
4      0.058973
5      0.008236
6      0.019217
7      0.007321
8      0.011185
9      0.009456
10     0.008338
11     0.016472
12     0.007626
13     0.007016
14     0.007117
15     0.010066
16     0.011286
17     0.007219
18     0.010168
19     0.020539
20     0.007931
21     0.033249
22     0.008643
23     0.013828
24     0.014743
25     0.016268
26     0.007728
27     0.007016
28     0.007728
29     0.088968
         ...   
564    0.008541
565    0.029995
566    0.016573
567    0.007016
568    0.008033
569    0.093950
570    0.015658
571    0.030605
572    0.008846
573    0.009354
574    0.011998
575    0.019624
576    0.008134
577    0.008744
578    0.013930
579    0.007219
580    0.026945
581    0.010168
582    0.024301
583    0.009659
584    0.007219
585    0.014947
586    0.007728
587    0.029893
588    0.033249
589    0.009049
590    0.010574
591    0.010269
592    0.007321
593    0.011795
Name: freq, Length: 594,

In [16]:
item_ser = dfp['items'] 

In [17]:
#creating a new dataframe 
dfp123 = pd.DataFrame()

In [18]:
dfp123

Empty DataFrame
Columns: []
Index: []

In [19]:
idx = 0

In [20]:
dfp123.insert(loc=idx, column='support', value=freq_ser_float)

In [21]:
dfp123.insert(loc=1, column='itemsets', value=item_ser)

In [22]:
dfp123

      support                                      itemsets
0    0.026029                                         [ham]
1    0.009151                       [ham, other-vegetables]
2    0.011490                             [ham, whole-milk]
3    0.011083                                      [liquor]
4    0.058973                                 [frankfurter]
5    0.008236                  [frankfurter, shopping-bags]
6    0.019217                     [frankfurter, rolls/buns]
7    0.007321                  [frankfurter, bottled-water]
8    0.011185                         [frankfurter, yogurt]
9    0.009456                 [frankfurter, tropical-fruit]
10   0.008338                         [frankfurter, pastry]
11   0.016472               [frankfurter, other-vegetables]
12   0.007626   [frankfurter, other-vegetables, whole-milk]
13   0.007016                  [frankfurter, domestic-eggs]
14   0.007117                    [frankfurter, brown-bread]
15   0.010066                        [fr

## 5. Calculation of different metrics:

In [29]:



def association_rules(df, metric="confidence", min_threshold=0.8):
    """Generates a DataFrame of association rules including the
    metrics 'score', 'confidence', and 'lift'
    Parameters
    -----------
    df : pandas DataFrame
      pandas DataFrame of frequent itemsets
      with columns ['support', 'itemsets']
    metric : string (default: 'confidence')
      Metric to evaluate if a rule is of interest.
      Supported metrics are 'support', 'confidence', 'lift'
      
      These metrics are computed as follows:
      - support(A->C) = support(A+C) [aka 'support'], range: [0, 1]
      - confidence(A->C) = support(A+C) / support(A), range: [0, 1]
      - lift(A->C) = confidence(A->C) / support(C), range: [0, inf]
     
    min_threshold : float (default: 0.8)
      Minimal threshold for the evaluation metric
      to decide whether a candidate rule is of interest.
    Returns
    ----------
    pandas DataFrame with columns "antecedent support",
      "consequent support",
      "support", "confidence", "lift",
      
      of all rules for which
      metric(rule) >= min_threshold.
    """

    # check for mandatory columns
    if not all(col in df.columns for col in ["support", "itemsets"]):
        raise ValueError("Dataframe needs to contain the\
                         columns 'support' and 'itemsets'")

   
    # metrics for association rules
    metric_dict = {
        "antecedent support": lambda _, sA, __: sA,
        "consequent support": lambda _, __, sC: sC,
        "support": lambda sAC, _, __: sAC,
        "confidence": lambda sAC, sA, _: sAC/sA,
        "lift": lambda sAC, sA, sC: metric_dict["confidence"](sAC, sA, sC)/sC,
        }

    columns_ordered = ["antecedent support", "consequent support",
                       "support",
                       "confidence", "lift"]
                       

    # check for metric compliance
    if metric not in metric_dict.keys():
        raise ValueError("Metric must be 'confidence' or 'lift', got '{}'"
                         .format(metric))

    # get dict of {frequent itemset} -> support
    keys = df['itemsets'].values
    values = df['support'].values
    frozenset_vect = np.vectorize(lambda x: frozenset(x))
    frequent_items_dict = dict(zip(frozenset_vect(keys), values))

    # prepare buckets to collect frequent rules
    rule_antecedants = []
    rule_consequents = []
    rule_supports = []

    # iterate over all frequent itemsets
    for k in frequent_items_dict.keys():
        sAC = frequent_items_dict[k]
        # to find all possible combinations
        for idx in range(len(k)-1, 0, -1):
            # of antecedent and consequent
            for c in combinations(k, r=idx):
                antecedent = frozenset(c)
                consequent = k.difference(antecedent)
                sA = frequent_items_dict[antecedent]
                sC = frequent_items_dict[consequent]
                # check for the threshold
                if metric_dict[metric](sAC, sA, sC) >= min_threshold:
                    rule_antecedants.append(antecedent)
                    rule_consequents.append(consequent)
                    rule_supports.append([sAC, sA, sC])

    # check if frequent rule was generated
    if not rule_supports:
        return pd.DataFrame(
            columns=["antecedants", "consequents"] + columns_ordered)
    else:
        # generate metrics
        rule_supports = np.array(rule_supports).T.astype(float)
        sAC = rule_supports[0]
        sA = rule_supports[1]
        sC = rule_supports[2]
        df_res = pd.DataFrame(
            data=list(zip(rule_antecedants, rule_consequents)),
            columns=["antecedants", "consequents"])
        for m in columns_ordered:
            df_res[m] = metric_dict[m](sAC, sA, sC)

    return df_res

In [24]:
#generating association rules and metrics for those rules
rules = association_rules(dfp123, metric="support", min_threshold=0.007)

In [25]:
#displaying results
rules.head()

                      antecedants                     consequents  \
0           (whole-milk, chicken)              (other-vegetables)   
1  (whole-milk, other-vegetables)                       (chicken)   
2     (chicken, other-vegetables)                    (whole-milk)   
3                    (whole-milk)     (chicken, other-vegetables)   
4                       (chicken)  (whole-milk, other-vegetables)   

   antecedent support  consequent support   support  confidence      lift  
0            0.017590            0.193493  0.008439    0.479769  2.479520  
1            0.074835            0.042908  0.008439    0.112772  2.628223  
2            0.017895            0.255516  0.008439    0.471591  1.845641  
3            0.255516            0.017895  0.008439    0.033028  1.845641  
4            0.042908            0.074835  0.008439    0.196682  2.628223  